In [1]:
import sys
sys.path.append('../')
from model.VQAE import VQAE
from model.MARL import MARL
from utils import device, add_noise
from tqdm import tqdm
import torch
from data.FloorPlanLoader import *
import torch.nn.functional as F
import random
import json
import torchvision
import matplotlib.pyplot as plt
import geopandas as gpd

USE_MULTISCALE = True
USE_MULTITASK = True

#Reproducability Checks:
random.seed(0) #Python
torch.manual_seed(0) #Torch
np.random.seed(0) #NumPy

import shutil
import torchvision
import torchvision.transforms.functional as TF
import imageio
from torchvision.utils import save_image
# %load_ext autoreload
# %autoreload 2
import csv
import numpy
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd
import seaborn as sns
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [2]:
#Hyperparameter
device = torch.device('cpu')
batch_size = 128
n_hiddens = 32
n_residual_hiddens = 32
n_residual_layers = 1
embedding_dim = 64
n_embeddings = 218
beta = .25
lr = 3e-3
epochs = 100
noise=False
noise_weight=0.05
img_channel=3 if USE_MULTISCALE else 1

In [3]:
curr_data = 'data00_left'

latent_file_path = f'../data/data_root/marl_latent_{curr_data}.pt'
latents = torch.load(latent_file_path)
floor = FloorPlanDataset(multi_scale=True, root=f'../data/data_root/{curr_data}/',\
                         data_config='../data/data_config/', preprocess=True)
data_loader = torch.utils.data.DataLoader(floor, batch_size=batch_size, shuffle=False)

In [5]:
n_clusters= 5

In [6]:
df_center_index = pd.read_csv(f'../results/MARL_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index

,label,index
0,1,1000
1,0,1001
2,2,1006
3,1,1007
4,2,1008
...,...,...
13972,1,993
13973,1,996
13974,0,997
13975,3,998


In [11]:
df_center_index_VQAE = pd.read_csv(f'../results/VQAE_{curr_data}_{n_clusters}clusters_clusterindex.csv', index_col=None)
df_center_index_VQAE

,label,index
0,0,1000
1,1,1001
2,4,1006
3,0,1007
4,4,1008
...,...,...
13972,0,993
13973,2,996
13974,1,997
13975,2,998


In [7]:
metadata_file_path = r"..\data\data_root\building_metadata_clean.csv"
metadata_df = pd.read_csv(metadata_file_path, index_col=False)
metadata_df.head(5)

,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,19,1981-2008 (2004),SingleFamily-2004,302.896371,315.9529,22.87
1,20,1981-2008 (2004),SingleFamily-2004,91.815805,153.5637,25.52
2,21,before 1980,SingleFamily-pre-1980,176.129465,200.8498,22.21
3,22,before 1980,SingleFamily-pre-1980,113.052927,153.0992,28.80
4,23,1981-2008 (2004),SingleFamily-2004,108.517313,152.4489,22.32


In [8]:
merged_df = pd.merge(df_center_index, metadata_df, left_on='index', right_on = 'OBJECTID', how='inner')
print(df_center_index.shape, merged_df.shape)
merged_df

(13977, 2) (7736, 8)


,label,index,OBJECTID,vintage,idf.kw,FootprintArea.m2,building.area.m2,HEIGHT
0,1,1007,1007,1981-2008 (2004),SingleFamily-2004,149.177364,227.6050,26.07
1,2,1008,1008,before 1980,SingleFamily-pre-1980,135.913344,103.0261,26.47
2,3,1010,1010,before 1980,MultiFamily-pre-1980,219.612523,332.3962,33.73
3,0,1012,1012,before 1980,SingleFamily-pre-1980,81.714012,117.7972,24.95
4,0,1015,1015,1981-2008 (2004),MultiFamily-2004,174.538913,231.5068,34.59
...,...,...,...,...,...,...,...,...
7731,3,982,982,before 1980,SingleFamily-pre-1980,222.303842,306.1055,24.54
7732,1,984,984,before 1980,SingleFamily-pre-1980,82.033143,59.6418,14.75
7733,3,987,987,before 1980,SingleFamily-pre-1980,199.989410,132.4754,22.75
7734,3,998,998,before 1980,MultiFamily-pre-1980,99.684203,88.2550,16.03


In [9]:
merged_df['idf.kw'].unique()

array(['SingleFamily-2004', 'SingleFamily-pre-1980',
       'MultiFamily-pre-1980', 'MultiFamily-2004', 'MultiFamily-2013',
       'SingleFamily-2013'], dtype=object)

In [10]:
# Separate 'SingleFamily' values
single_family_df = merged_df[merged_df['idf.kw'].str.contains('SingleFamily')]
print(single_family_df.shape)
# Separate 'MultiFamily' values
multi_family_df = merged_df[merged_df['idf.kw'].str.contains('MultiFamily')]
print(multi_family_df.shape)

(5317, 8)
(2419, 8)


In [ ]:
single_family_df[['index']].to_csv('..notebooks/categorical_clustering/data00_left_singlefamily.csv')
multi_family_df[['index']].to_csv('..notebooks/categorical_clustering/data00_left_multifamily.csv')

In [26]:
csv_file = f'../notebooks/categorical_clustering/data00_left_multifamily.csv'
# Read the CSV file into a dataframe
df = pd.read_csv(csv_file)
df

,Unnamed: 0,index
0,2,1010
1,4,1015
2,5,1019
3,9,1029
4,10,1035
...,...,...
2414,7724,966
2415,7725,967
2416,7727,972
2417,7730,980


In [27]:

# Assuming your CSV file has a column named 'OBJECTID' and the images are in the format 'f{objectid}.png'
curr_data_folder = f'../data/data_root/{curr_data}/'
output_folder = f'../data/data_root/{curr_data}_MF/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
num_files_copied = 0

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    object_id = row['index']
    image_file = f'{object_id}.png'
    pt_file = f'{object_id}.pt'
    img_source_path = os.path.join(curr_data_folder, image_file)
    img_destination_path = os.path.join(output_folder, image_file)
    
    pt_source_path = os.path.join(curr_data_folder, pt_file)
    pt_destination_path = os.path.join(output_folder, pt_file)
    
    if os.path.exists(img_source_path):
        shutil.copy(img_source_path, img_destination_path)
        shutil.copy(pt_source_path, pt_destination_path)
        num_files_copied += 1

# Display the total number of files copied
print(f"Total number of files copied: {num_files_copied}")

Total number of files copied: 2419
